pip install fastapi uvicorn pandas pyarrow


In [ ]:
from fastapi import FastAPI
import pandas as pd

# Cargar el archivo Parquet
combined_df = pd.read_parquet('/mnt/data/combined_df.parquet')

# Crear la aplicación FastAPI
app = FastAPI()

# Función para traducir meses y días de español a inglés (para usarlos con pandas)
month_translation = {
    'enero': 'January', 'febrero': 'February', 'marzo': 'March', 'abril': 'April',
    'mayo': 'May', 'junio': 'June', 'julio': 'July', 'agosto': 'August',
    'septiembre': 'September', 'octubre': 'October', 'noviembre': 'November', 'diciembre': 'December'
}

day_translation = {
    'lunes': 'Monday', 'martes': 'Tuesday', 'miércoles': 'Wednesday',
    'jueves': 'Thursday', 'viernes': 'Friday', 'sábado': 'Saturday', 'domingo': 'Sunday'
}

# Endpoint: Cantidad de filmaciones por mes
@app.get('/cantidad_filmaciones_mes/{mes}')
def cantidad_filmaciones_mes(mes: str):
    mes_ingles = month_translation.get(mes.lower(), None)
    if mes_ingles:
        count = combined_df[combined_df['release_date'].dt.strftime('%B') == mes_ingles].shape[0]
        return f"{count} cantidad de películas fueron estrenadas en el mes de {mes.capitalize()}"
    return f"Mes '{mes}' no encontrado."

# Endpoint: Cantidad de filmaciones por día
@app.get('/cantidad_filmaciones_dia/{dia}')
def cantidad_filmaciones_dia(dia: str):
    dia_ingles = day_translation.get(dia.lower(), None)
    if dia_ingles:
        count = combined_df[combined_df['release_date'].dt.strftime('%A') == dia_ingles].shape[0]
        return f"{count} cantidad de películas fueron estrenadas en los días {dia.capitalize()}"
    return f"Día '{dia}' no encontrado."

# Endpoint: Score de una filmación por título
@app.get('/score_titulo/{titulo}')
def score_titulo(titulo: str):
    film = combined_df[combined_df['title'].str.lower() == titulo.lower()]
    if not film.empty:
        title = film['title'].values[0]
        release_year = film['release_year'].values[0]
        score = film['popularity'].values[0]
        return f"La película {title} fue estrenada en el año {release_year} con un score/popularidad de {score}"
    return f"Película '{titulo}' no encontrada."

# Endpoint: Votos de una filmación por título
@app.get('/votos_titulo/{titulo}')
def votos_titulo(titulo: str):
    film = combined_df[combined_df['title'].str.lower() == titulo.lower()]
    if not film.empty:
        vote_count = film['vote_count'].values[0]
        if vote_count >= 2000:
            vote_average = film['vote_average'].values[0]
            title = film['title'].values[0]
            release_year = film['release_year'].values[0]
            return f"La película {title} fue estrenada en el año {release_year}. Cuenta con un total de {vote_count} valoraciones, con un promedio de {vote_average}"
        return f"La película '{titulo}' no cumple con la condición de al menos 2000 valoraciones."
    return f"Película '{titulo}' no encontrada."

# Endpoint: Información sobre un actor
@app.get('/get_actor/{nombre_actor}')
def get_actor(nombre_actor: str):
    films = combined_df[combined_df['cast'].str.contains(nombre_actor, case=False, na=False)]
    if not films.empty:
        total_films = films.shape[0]
        total_return = films['return'].sum()
        avg_return = films['return'].mean()
        return f"El actor {nombre_actor} ha participado en {total_films} filmaciones. Ha conseguido un retorno de {total_return} con un promedio de {avg_return} por filmación"
    return f"Actor '{nombre_actor}' no encontrado."

# Endpoint: Información sobre un director
@app.get('/get_director/{nombre_director}')
def get_director(nombre_director: str):
    films = combined_df[combined_df['crew'].str.contains(nombre_director, case=False, na=False) & (combined_df['job'] == 'Director')]
    if not films.empty:
        director_info = []
        for _, film in films.iterrows():
            title = film['title']
            release_date = film['release_date']
            revenue = film['revenue']
            budget = film['budget']
            individual_return = film['return']
            director_info.append({
                'title': title,
                'release_date': release_date,
                'budget': budget,
                'revenue': revenue,
                'return': individual_return
            })
        return {
            'director': nombre_director,
            'films': director_info
        }
    return f"Director '{nombre_director}' no encontrado."
